# Concepts

```{note}
The package is in the alpha phase of development. This notebook illustrates the concepts and a current state of development. 
Anything you see here may, and likely will, change.
```

The goal is to mimic what stars package in R can do - [https://r-spatial.github.io/stars/articles/stars1.html#vector-data-cube-example](https://r-spatial.github.io/stars/articles/stars1.html#vector-data-cube-example).

![vector data cube illustration](https://raw.githubusercontent.com/r-spatial/stars/master/images/cube3.png)

Because we need to use geometries as an index, we need to ensure that geopandas is using shapely 2.0 (beta) as a geometry engine. Shapely 1.8 geometries are not hashable, while shapely 2.0 are.

In [1]:
import geopandas
import shapely
import pandas
import xarray
import numpy

from xvec import GeoVectorIndex

We want to recreate [this example from stars documentation](https://r-spatial.github.io/stars/articles/stars1.html#vector-data-cube-example).

Load geometries using geopandas:

In [2]:
nc = geopandas.read_file("https://github.com/r-spatial/sf/raw/main/inst/gpkg/nc.gpkg")

Get the GeometryArray. Note that this also contains CRS information at this point.

In [3]:
origin = destination = nc.geometry.array

Create dimensions and dummy data.

In [4]:
mode = ["car", "bike", "foot"]
day = pandas.date_range("2015-01-01", periods=100)
hours = range(24)
data = numpy.random.randint(1, 100, size=(3, 100, 24, 100, 100))

Create `xarray.DataArray`.

In [5]:
arr = xarray.DataArray(data, coords=(mode, day, hours, origin, destination), dims=["mode", "day", "time", "origin", "destination"])
arr

<xarray.DataArray (mode: 3, day: 100, time: 24, origin: 100, destination: 100)>
array([[[[[ 3, 20, 86, ..., 47, 18,  4],
          [60, 27, 15, ..., 54,  1, 23],
          [65, 21, 14, ..., 70, 97, 71],
          ...,
          [75, 34, 41, ..., 79,  7, 68],
          [31, 64, 18, ..., 44,  9, 37],
          [35, 71, 59, ..., 66, 71,  1]],

         [[ 8, 54, 27, ..., 79, 20, 93],
          [ 9, 94, 80, ..., 69, 25, 78],
          [78, 86, 59, ..., 24, 68, 13],
          ...,
          [92, 73, 72, ..., 55, 75, 86],
          [39, 40, 55, ..., 63, 24, 64],
          [63, 85, 17, ..., 44, 75, 68]],

         [[24, 54, 24, ..., 99, 40, 89],
          [32, 39, 46, ..., 38, 21,  3],
          [39, 63, 90, ..., 54,  5, 84],
          ...,
...
          ...,
          [91, 56, 61, ..., 91, 73, 25],
          [37, 91, 54, ..., 47, 72, 37],
          [72, 81, 93, ..., 59, 84, 68]],

         [[59, 76, 62, ..., 84, 38, 60],
          [11, 41, 61, ..., 70, 50, 98],
          [66, 59, 55, ..., 19, 21, 86],
          ...,
          [41, 49, 95, ..., 28, 87, 23],
          [25, 52,  4, ..., 18, 78, 90],
          [93,  1, 48, ..., 22, 48, 45]],

         [[96, 94, 67, ..., 72, 32, 20],
          [59, 71, 78, ..., 92, 29, 23],
          [19, 56, 11, ..., 34, 30, 82],
          ...,
          [86, 83, 55, ..., 66, 67, 17],
          [27, 19, 10, ..., 42, 55, 73],
          [11, 25, 62, ..., 40, 84, 95]]]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
  * day          (day) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-04-10
  * time         (time) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * origin       (origin) object MULTIPOLYGON (((-81.4727554321289 36.2343559...
  * destination  (destination) object MULTIPOLYGON (((-81.4727554321289 36.23...

Set two `GeoVectorIndex` instances for the `origin` and `destination` coordinates, respectively (first drop the two `pandas.Index` objects that were set by default for these two dimension coordinates).

In [6]:
arr = (
    arr
    .drop_indexes(["origin", "destination"])
    .set_xindex("origin", GeoVectorIndex, crs=origin.crs)
    .set_xindex("destination", GeoVectorIndex, crs=destination.crs)
)

arr

<xarray.DataArray (mode: 3, day: 100, time: 24, origin: 100, destination: 100)>
array([[[[[ 3, 20, 86, ..., 47, 18,  4],
          [60, 27, 15, ..., 54,  1, 23],
          [65, 21, 14, ..., 70, 97, 71],
          ...,
          [75, 34, 41, ..., 79,  7, 68],
          [31, 64, 18, ..., 44,  9, 37],
          [35, 71, 59, ..., 66, 71,  1]],

         [[ 8, 54, 27, ..., 79, 20, 93],
          [ 9, 94, 80, ..., 69, 25, 78],
          [78, 86, 59, ..., 24, 68, 13],
          ...,
          [92, 73, 72, ..., 55, 75, 86],
          [39, 40, 55, ..., 63, 24, 64],
          [63, 85, 17, ..., 44, 75, 68]],

         [[24, 54, 24, ..., 99, 40, 89],
          [32, 39, 46, ..., 38, 21,  3],
          [39, 63, 90, ..., 54,  5, 84],
          ...,
...
          ...,
          [91, 56, 61, ..., 91, 73, 25],
          [37, 91, 54, ..., 47, 72, 37],
          [72, 81, 93, ..., 59, 84, 68]],

         [[59, 76, 62, ..., 84, 38, 60],
          [11, 41, 61, ..., 70, 50, 98],
          [66, 59, 55, ..., 19, 21, 86],
          ...,
          [41, 49, 95, ..., 28, 87, 23],
          [25, 52,  4, ..., 18, 78, 90],
          [93,  1, 48, ..., 22, 48, 45]],

         [[96, 94, 67, ..., 72, 32, 20],
          [59, 71, 78, ..., 92, 29, 23],
          [19, 56, 11, ..., 34, 30, 82],
          ...,
          [86, 83, 55, ..., 66, 67, 17],
          [27, 19, 10, ..., 42, 55, 73],
          [11, 25, 62, ..., 40, 84, 95]]]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
  * day          (day) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-04-10
  * time         (time) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * origin       (origin) object MULTIPOLYGON (((-81.4727554321289 36.2343559...
  * destination  (destination) object MULTIPOLYGON (((-81.4727554321289 36.23...
Indexes:
    origin       GeoVectorIndex(crs=EPSG:4267)
    destination  GeoVectorIndex(crs=EPSG:4267)

In [7]:
arr.xindexes

Indexes:
    mode         PandasIndex
    day          PandasIndex
    time         PandasIndex
    origin       GeoVectorIndex(crs=EPSG:4267)
    destination  GeoVectorIndex(crs=EPSG:4267)

Access to the CRS via the index:

In [8]:
arr.xindexes["origin"].crs

<Geographic 2D CRS: EPSG:4267>
Name: NAD27
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North and central America: Antigua and Barbuda - onshore. Bahamas - onshore plus offshore over internal continental shelf only. Belize - onshore. British Virgin Islands - onshore. Canada onshore - Alberta, British Columbia, Manitoba, New Brunswick, Newfoundland and Labrador, Northwest Territories, Nova Scotia, Nunavut, Ontario, Prince Edward Island, Quebec, Saskatchewan and Yukon - plus offshore east coast. Cuba - onshore and offshore. El Salvador - onshore. Guatemala - onshore. Honduras - onshore. Panama - onshore. Puerto Rico - onshore. Mexico - onshore plus offshore east coast. Nicaragua - onshore. United States (USA) onshore and offshore - Alabama, Alaska, Arizona, Arkansas, California, Colorado, Connecticut, Delaware, Florida, Georgia, Idaho, Illinois, Indiana, Iowa, Kansas, Kentucky, Louisiana, Maine, Maryland, 

Default data selection using the "nearest" mode.

In [9]:
arr.sel(
    day="2015-01-01",
    time=0,
    origin=[shapely.Point(-80, 35), shapely.Point(-76, 35.6)],
    destination=[shapely.Point(-81, 36), shapely.Point(-77, 35.5)],
    method="nearest",
)

<xarray.DataArray (mode: 3, origin: 2, destination: 2)>
array([[[40, 72],
        [95, 66]],

       [[66, 79],
        [81, 33]],

       [[80, 42],
        [56, 11]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
    day          datetime64[ns] 2015-01-01
    time         int64 0
  * origin       (origin) object MULTIPOLYGON (((-79.91995239257812 34.807918...
  * destination  (destination) object MULTIPOLYGON (((-80.72651672363281 35.5...
Indexes:
    origin       GeoVectorIndex(crs=EPSG:4267)
    destination  GeoVectorIndex(crs=EPSG:4267)

Alternative data selection using the "query" mode with a single geometry and a given predicate:

In [10]:
arr.sel(origin=shapely.box(-80, 35, -77, 38), method="intersects")

<xarray.DataArray (mode: 3, day: 100, time: 24, origin: 41, destination: 100)>
array([[[[[63,  4, 52, ..., 94, 53, 42],
          [60, 43, 49, ..., 98, 60, 21],
          [60, 58, 57, ..., 46, 75, 81],
          ...,
          [82,  6, 34, ..., 72, 90, 61],
          [87, 53, 91, ..., 80, 47, 39],
          [24, 77, 60, ...,  3, 95, 81]],

         [[84, 17, 14, ..., 78, 62, 16],
          [82, 90, 50, ..., 70, 78, 60],
          [57, 43, 52, ...,  5, 15, 11],
          ...,
          [58, 55, 49, ..., 99, 88, 42],
          [ 2, 47, 44, ..., 78, 25, 75],
          [10,  8, 35, ..., 26, 60, 50]],

         [[46, 29, 56, ..., 65, 28, 10],
          [67, 50, 25, ..., 30, 18, 60],
          [45, 87, 92, ..., 19, 41, 79],
          ...,
...
          ...,
          [30, 28, 32, ...,  4, 11, 23],
          [92, 91, 99, ..., 79, 56, 52],
          [10, 94, 41, ...,  8, 26, 88]],

         [[27, 89, 88, ..., 96, 13, 41],
          [25, 36, 66, ..., 91, 92, 45],
          [13, 94, 11, ..., 94, 22, 83],
          ...,
          [53, 51, 69, ..., 27, 17, 89],
          [37, 14, 40, ..., 46, 46, 93],
          [89, 15,  9, ..., 46, 49, 71]],

         [[70, 43, 47, ..., 99, 28, 59],
          [35, 68, 47, ..., 91,  1, 38],
          [90, 12, 13, ..., 54, 55, 69],
          ...,
          [93, 27, 67, ..., 96, 80, 67],
          [16,  9, 21, ..., 53, 59, 72],
          [14, 67, 89, ..., 22, 23, 36]]]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
  * day          (day) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-04-10
  * time         (time) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * origin       (origin) object MULTIPOLYGON (((-78.11376953125 34.720985412...
  * destination  (destination) object MULTIPOLYGON (((-81.4727554321289 36.23...
Indexes:
    origin       GeoVectorIndex(crs=EPSG:4267)
    destination  GeoVectorIndex(crs=EPSG:4267)

"Query" mode selection with an array-like of geometries is not supported:

In [11]:
arr.sel(origin=[shapely.Point(-80, 35), shapely.Point(-76, 35.6)], method="intersects")

ValueError: selection with another method than nearest only supports a single geometry as input label.

Other operations like `isel`, `concat` and alignment work just like with any default (pandas) index:

In [12]:
arr + arr.isel(origin=[0, 1], destination=[-2, -1])

<xarray.DataArray (mode: 3, day: 100, time: 24, origin: 2, destination: 2)>
array([[[[[ 36,   8],
          [  2,  46]],

         [[ 40, 186],
          [ 50, 156]],

         [[ 80, 178],
          [ 42,   6]],

         ...,

         [[192,  20],
          [ 78,  68]],

         [[ 58, 102],
          [112,  90]],

         [[ 62, 106],
          [138,  64]]],

...

        [[[  4, 166],
          [146,  80]],

         [[ 70, 166],
          [138, 166]],

         [[128, 108],
          [150, 178]],

         ...,

         [[186,  68],
          [ 58, 132]],

         [[ 76, 120],
          [100, 196]],

         [[ 64,  40],
          [ 58,  46]]]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
  * day          (day) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-04-10
  * time         (time) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * origin       (origin) object MULTIPOLYGON (((-81.4727554321289 36.2343559...
  * destination  (destination) object MULTIPOLYGON (((-77.96073150634766 34.1...
Indexes:
    origin       GeoVectorIndex(crs=EPSG:4267)
    destination  GeoVectorIndex(crs=EPSG:4267)

In [13]:
xarray.concat([arr.isel(origin=[0, 1]), arr.isel(origin=[-2, -1])], dim="origin")

<xarray.DataArray (mode: 3, day: 100, time: 24, origin: 4, destination: 100)>
array([[[[[ 3, 20, 86, ..., 47, 18,  4],
          [60, 27, 15, ..., 54,  1, 23],
          [31, 64, 18, ..., 44,  9, 37],
          [35, 71, 59, ..., 66, 71,  1]],

         [[ 8, 54, 27, ..., 79, 20, 93],
          [ 9, 94, 80, ..., 69, 25, 78],
          [39, 40, 55, ..., 63, 24, 64],
          [63, 85, 17, ..., 44, 75, 68]],

         [[24, 54, 24, ..., 99, 40, 89],
          [32, 39, 46, ..., 38, 21,  3],
          [55, 69, 74, ..., 54, 53, 68],
          [30, 21, 44, ..., 36, 31, 62]],

         ...,

         [[38, 39, 62, ..., 80, 96, 10],
          [39, 92, 83, ..., 50, 39, 34],
          [70, 31, 68, ..., 91, 32, 15],
...
          [67,  4, 40, ..., 36, 75, 89],
          [ 8, 27, 47, ..., 46, 60,  6],
          [94,  4, 66, ...,  1, 72, 94]],

         ...,

         [[59, 95, 17, ...,  2, 93, 34],
          [56, 54, 73, ..., 81, 29, 66],
          [37, 91, 54, ..., 47, 72, 37],
          [72, 81, 93, ..., 59, 84, 68]],

         [[59, 76, 62, ..., 84, 38, 60],
          [11, 41, 61, ..., 70, 50, 98],
          [25, 52,  4, ..., 18, 78, 90],
          [93,  1, 48, ..., 22, 48, 45]],

         [[96, 94, 67, ..., 72, 32, 20],
          [59, 71, 78, ..., 92, 29, 23],
          [27, 19, 10, ..., 42, 55, 73],
          [11, 25, 62, ..., 40, 84, 95]]]]])
Coordinates:
  * mode         (mode) <U4 'car' 'bike' 'foot'
  * day          (day) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-04-10
  * time         (time) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * origin       (origin) object MULTIPOLYGON (((-81.4727554321289 36.2343559...
  * destination  (destination) object MULTIPOLYGON (((-81.4727554321289 36.23...
Indexes:
    origin       GeoVectorIndex(crs=EPSG:4267)
    destination  GeoVectorIndex(crs=EPSG:4267)